# Bibliotecas usadas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,roc_curve, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,cross_validate

# Exploração dos dados

## Analisando a estrura do dataframe

In [ ]:
df = pd.read_excel('default_of_credit_card_clients__courseware_version_1_21_19.xls')

In [ ]:
print(f'A Quantidade de linhas nesse data frame é {df.shape[0]}. Enquanto a quantidade de colunas é de {df.shape[1]}')

In [ ]:
print(f'Temos um total de {sum(df.dtypes == "int64")} variáveis inteiras e {sum(df.dtypes == "object")} do tipo objeto')

Ao analisar as informações acima podemos observar que a quantidade dos dados presentes no dataframe está de acordo com o dicionário dos dados

## Verificando se na coluna "id" existe valores duplicados

In [ ]:
print(f'A quantidade de valores duplicados é de {(df.shape[0])-(df["ID"].nunique())}')

### Analisando os dados que estão duplicados

In [ ]:
id_counts = df['ID'].value_counts()
duplicados = id_counts == 2
duplicados = id_counts.index[duplicados]
print(f'Verificando a quantidade de duplicados {len(duplicados)}.')
df.loc[df['ID'].isin(duplicados),:].head(5)

Todos os id's que estão duplicados tem, em alguma coluna, valores igual a zero. Sendo que em algumas delas esses zeros não deveria existir. Então não faz sentido manter esses valores.

### Excluindo as linhas que tem, em todas as colunas, valores igual a zero

In [ ]:
sem_zeros = df == 0
zeros_mask = sem_zeros.iloc[:,1:].all(axis=1)
df_clean_1 = df.loc[~zeros_mask,:].copy()
print(f'A quantidade de valores unicos era de {df["ID"].nunique()}')
print(f'Sem os valores duplicados, a quantidade de dados agora é de {len(df_clean_1)}')

## Analisando e modificando os dados categóricos

### 1. SEX

A variável "SEX" não vai ser utilizada

In [ ]:
df_clean_1.drop('SEX', axis=1, inplace=True)

### 2. EDUCATION

In [ ]:
df_clean_1['EDUCATION'].value_counts()

A variável EDUCATION tem, segundo as informações do dicionário, as variáveis: 1 = pós-graduação; 2 = universidade; 3 = ensino médio; 4 = outros.

Portanto, não faz sentindo manter outros valores diferentes desse. Assim, o outros valores serão adicionados no número 4, "outros"...

In [ ]:
df_clean_1['EDUCATION'].replace(to_replace=[0,5,6], value = 4, inplace=True)

In [ ]:
df_clean_1['EDUCATION'].value_counts()

### 3. MARRIAGE

In [ ]:
df_clean_1['MARRIAGE'].value_counts()

Quase que da mesma forma que a variável anterior, tem um valor que não identificado ao analisar o dicionário de dados.

Assim sendo, o 1 = Casado; 2 = Solteiro ; 3 = Outros.

O valor 4 não é identificado no dicionário

In [ ]:
df_clean_1['MARRIAGE'].replace(to_replace = 4, value=3, inplace=True)

In [ ]:
df_clean_1['MARRIAGE'].value_counts()

### 4. Mapeando os dados categoricos

#### EDUCATION

In [ ]:
edu_map = {
    1:'pos-graduacao',
    2:'graduacao',
    3:'medio',
    4:'outros'
}

df_clean_1['EDU_CAT'] = df_clean_1['EDUCATION'].map(edu_map)
edu_ohe = pd.get_dummies(df_clean_1['EDU_CAT'])
df_clean_1 = pd.concat([df_clean_1, edu_ohe], axis=1)
df_clean_1[['EDUCATION','EDU_CAT','pos-graduacao','graduacao','medio','outros']].head(5)

#### MARRIAGE

In [ ]:
df_clean_1.MARRIAGE.value_counts()

mar_map = {
    1:'casado',
    2:'solteiro',
    3:'outros'
}

df_clean_1['MAR_CAT'] = df_clean_1['MARRIAGE'].map(mar_map)
df_clean_1['MAR_CAT'].value_counts()
mar_ohe = pd.get_dummies(df_clean_1['MAR_CAT'])
df_clean_1 = pd.concat([df_clean_1, mar_ohe], axis=1)
df_clean_1[['MARRIAGE', 'MAR_CAT','casado','solteiro','outros']].head(5)

## Analisando dados continuos

### Analisando os dados de pagamento

#### Modificando a coluna "PAY_1"

In [ ]:
df_clean_1['PAY_1'].value_counts()

In [ ]:
pay_clean = df_clean_1['PAY_1'] != 'Not available'
df_clean_2 = df_clean_1.loc[pay_clean,:].copy()
df_clean_2['PAY_1'] = df_clean_2['PAY_1'].astype('int64')

In [ ]:
df_clean_2[['PAY_1','PAY_2']].info()

In [ ]:
df_clean_2['PAY_1'].value_counts()

### Analise geral dos dados de pagamento

#### 1. PAY

In [ ]:
serie_pagamento = ["PAY_1","PAY_2",'PAY_3','PAY_4',"PAY_5","PAY_6"]
df_clean_2[serie_pagamento].hist(bins=np.array(range(-2,10)), layout=(2,3))

Aparentemente somente a coluna PAY_1 será valida para a construção do modelo. Isso se deve ao fato de, ao analisar os gráficos, perceber que todas as outras variáveis são, aproximadamente, igual. 

In [ ]:
df_clean_3 = df_clean_2.drop(["PAY_2",'PAY_3','PAY_4',"PAY_5","PAY_6"], axis=1).copy()

#### 2. BILL_PAY

In [ ]:
bills = ['BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']

df_clean_3[bills].apply(np.log10).describe()

In [ ]:
#df_clean_3[bills].apply(np.log10).hist(bins=20, layout=(2,3))

In [ ]:
df_clean_3[bills].apply(np.log10).plot.box()

O que faria a pessoa ter um saldo de crédito negativo? O banco está devendo para ela?

#### 3. PAY AMT

In [ ]:
pay_amt =['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

df_clean_3[pay_amt].apply(np.log10).describe()

In [ ]:
#df_clean_3[pay_amt].apply(np.log10).hist(bins=20,layout=(2,3),ylabelsize=8)

In [ ]:
df_clean_3[pay_amt].apply(np.log10).plot.box()

### Salvando o data frame

In [ ]:
df_clean_2.to_csv('df_2.csv', index=False)

# Regressão logística

In [ ]:
x = df_clean_2[['PAY_1','LIMIT_BAL']]
y = df_clean_2['default payment next month']

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, random_state=1, test_size=0.2)

In [ ]:
log_rg = LogisticRegression(solver='liblinear')
log_rg.fit(x_treino, y_treino)
y_pred = log_rg.predict(x_teste)
y_pred_prob = log_rg.predict_proba(x_teste)

In [ ]:
# Calculando as probabilidades dos valores manualmente
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y

coef_int = np.concatenate([log_rg.intercept_.reshape(1,1), log_rg.coef_], axis=1)
ones = np.hstack([np.ones((x_teste.shape[0],1)), x_teste])
x_lin_comb = np.dot(coef_int, np.transpose(ones))
y_pred_prob_manual = sigmoid(x_lin_comb)
y_pred_manual = y_pred_prob_manual >= 0.5

# Verificando se os array que foi calculado manualmente ou através da função são iguais
print(np.array_equal(y_pred.reshape(1,-1), y_pred_manual))

# Calculandoo as ROC AUC's
roc_auc_score(y_true=y_teste,y_score=y_pred_prob[:,1])

# Quanto mais proximo de 0.5 pior é o modelo, enquanto que quanto mais perto de 1 melhor. Nesse caso, tem-se muito o que melhorar...

# Validação cruzada, regularização e tunagem

In [ ]:
# instanciando o modelo de Reg logistica
new_log_rg = LogisticRegression(solver='saga', penalty = 'l1', max_iter = 1000)

# instanciando o fold
k_folds = StratifiedKFold(n_splits=4, shuffle=True, random_state=1)

# instanciando o MinMaxScaler
min_max_sc = MinMaxScaler()

# Criando a pipeline
scaler_lr_pipeline = Pipeline(steps = [('scaler', min_max_sc), ('model', new_log_rg)])

In [ ]:
def cross_val_C_search_pipe(k_folds, C_vals, pipeline, X, Y):
    
    n_folds = k_folds.n_splits
    cv_train_roc_auc = np.empty((n_folds, len(C_vals)))
    cv_test_roc_auc = np.empty((n_folds, len(C_vals)))
    cv_test_roc = [[]]*len(C_vals)

    for c_val_counter in range(len(C_vals)):
        #Set the C value for the model object
        pipeline.set_params(model__C = C_vals[c_val_counter])
        #Count folds for each value of C
        fold_counter = 0
        #Get training and testing indices for each fold
        for train_index, test_index in k_folds.split(X, Y):
            #Subset the features and response, for training and testing data for
            #this fold
            X_cv_train, X_cv_test = X[train_index], X[test_index]
            y_cv_train, y_cv_test = Y[train_index], Y[test_index]

            #Fit the model on the training data
            pipeline.fit(X_cv_train, y_cv_train)

            #Get the training ROC AUC
            y_cv_train_predict_proba = pipeline.predict_proba(X_cv_train)
            cv_train_roc_auc[fold_counter, c_val_counter] = \
            roc_auc_score(y_cv_train, y_cv_train_predict_proba[:,1])

            #Get the testing ROC AUC
            y_cv_test_predict_proba = pipeline.predict_proba(X_cv_test)
            cv_test_roc_auc[fold_counter, c_val_counter] = \
            roc_auc_score(y_cv_test, y_cv_test_predict_proba[:,1])

            #Testing ROC curves for each fold
            this_fold_roc = roc_curve(y_cv_test, y_cv_test_predict_proba[:,1])
            cv_test_roc[c_val_counter].append(this_fold_roc)

            #Increment the fold counter
            fold_counter += 1

        #Indicate progress
        print('Done with C = {}'.format(pipeline.get_params()['model__C']))

    return cv_train_roc_auc, cv_test_roc_auc, cv_test_roc

In [ ]:
# Selecionando as variáveis
x = df_clean_3[['LIMIT_BAL',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_1',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6']]

y = df_clean_3['default payment next month'].replace(to_replace=4,value=0)

# Separando o dataset
x_treino, x_teste, y_treino, y_teste = train_test_split(x.values, y.values, random_state=1, test_size=0.2)

In [ ]:
#cv_train_roc_auc, cv_test_roc_auc, cv_test_roc = cross_val_C_search_pipe(k_folds=k_folds, C_vals=c_values, pipeline = scaler_lr_pipeline, X = x_treino, Y=y_treino)

In [ ]:
#plt.plot([2,1,0,-1,-2,-3],np.mean(cv_train_roc_auc, axis=0), '-o', label = 'treino')
#plt.plot([2,1,0,-1,-2,-3],np.mean(cv_test_roc_auc, axis=0), '-x', label = 'teste')
#plt.ylabel('ROC AUC')
#plt.xlabel('log$_{10}$(C)')
#plt.legend()

# Pouca diferença entre os valores, ou seja, o over ou under fit "não existe"

## Criando caracteristicas de intereção

In [ ]:
# Instacioando o elemento e fazendo as interações
interacoes = PolynomialFeatures(degree=2,interaction_only=True, include_bias = False)
f_interacoes = interacoes.fit_transform(x.values)

# Separando os dados
x_treino, x_teste, y_treino, y_teste = train_test_split(f_interacoes, y.values, random_state=1, test_size=0.2)

In [ ]:
# Refazendo o teste com os folds
# comentado para que não haja necessidade de rodar toda vez, já que é uma etapa que demora
#cv_train_roc_auc, cv_test_roc_auc, cv_test_roc = cross_val_C_search_pipe(k_folds=k_folds, C_vals=c_values, pipeline = scaler_lr_pipeline, X = x_treino, Y=y_treino)


# Construindo o gráfico com as novas features
#plt.plot([2,1,0,-1,-2,-3],np.mean(cv_train_roc_auc, axis=0), '-o', label = 'treino')
#plt.plot([2,1,0,-1,-2,-3],np.mean(cv_test_roc_auc, axis=0), '-x', label = 'teste')
#plt.ylabel('ROC AUC')
#plt.xlabel('log$_{10}$(C)')
#plt.legend()


# Pouca diferença com essas novas features...

# Árvore de decisão

In [ ]:
# Instaciando o modelo
dt = tree.DecisionTreeClassifier(max_depth=2)

# Dados que serão utilizados
x = df_clean_3[['LIMIT_BAL', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']]

y = df_clean_3[['default payment next month']]

# Separação dos dados
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y,test_size=0.2,random_state=1)

# Treinando o modelo
dt = dt.fit(x_treino, y_treino)

# Visualizando o gráfico criado
dt_data = tree.export_graphviz(dt, out_file=None, filled=True, rounded=True, feature_names=x.columns, 
                               proportion=True,class_names=['Not defaulted', 'Defaulted'])

# Renderizando a imagem
graphviz.Source(dt_data)

# Florestas aleatórias

In [ ]:
# Instaciando o modelo
rf = RandomForestClassifier()

# Configurando os parametros para ser testados
params = {'max_depth':[3,6,9,12], 'n_estimators':[50,100,200]}

# Instaciando o gridsearch e treinando
grid_cv = GridSearchCV(rf, param_grid=params, scoring='roc_auc', cv=4,verbose=1, pre_dispatch=None,error_score=np.nan, return_train_score=True)
grid_cv.fit(x_treino, y_treino)

# Melhores paramatros para esses dados
grid_cv.best_params_

# Preenchendo os valores faltantes do dataset inicial para retreino

### Criando um modelo para prever os dados faltantes

In [ ]:
# Ajustando os dados
y = x['PAY_1']
x = x.drop('PAY_1', axis=1)

In [ ]:
# Dividindo os dados
x_treino, x_teste, y_treino, y_teste = train_test_split(x.values,y,test_size=0.2, random_state=1)

# Parametros que serão testados
rf_params = {'max_depth':[3,6,9,12], 'n_estimators':[10,50,100,200]}

# Instanciando o floresta randomica
rf = RandomForestClassifier()

# Instanciando e executando o GridSeachCV
cv_rf_input = GridSearchCV(rf,param_grid=rf_params, scoring='accuracy', n_jobs=1, refit=True,cv=4,
                           verbose=2,error_score=np.nan, return_train_score=True)
cv_rf_input.fit(x_treino, y_treino)

# Melhores parametros e resultado para esse modelo
display(cv_rf_input.best_params_)
display(cv_rf_input.best_score_)

In [ ]:
# Verificando se a acurácia está correta com o valor dos dados de teste
y_impute_predict = cv_rf_input.predict(x_teste)
accuracy_score(y_pred=y_impute_predict, y_true=y_teste)

# Os valores são compativeis, ou seja, não está overfitado ou underfitado.

In [ ]:
# Gerando gráfico dessa previsão para ver se há compatibilidade
fig, axs = plt.subplots(1,2, figsize=(8,3))
axs[0].hist(y_teste)
axs[0].set_title('Pay 1 sem valores faltantes')
axs[1].hist(y_impute_predict)
axs[1].set_title('Pay 1 do modelo (com dados inputados)')
plt.tight_layout()

### Prevendo os valores

In [ ]:
# Instancionando um novo modelo com os parametros ajustados e treinando-o com todos os dados disponveis para prever a variável pay_1
rf_imput = RandomForestClassifier(n_estimators=100, max_depth=12)
rf_imput.fit(df_clean_3[['LIMIT_BAL', 'EDUCATION', 'MARRIAGE', 'AGE', 'BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']], df_clean_3['PAY_1'])

In [ ]:
# Separando o dataframe para só ter os valores que precisão ser ajustados
na_pay1 = df_clean_1['PAY_1'] == 'Not available'
df_pay1_na = df_clean_1.loc[na_pay1,:]
df_pay1_na = df_pay1_na[['LIMIT_BAL', 'EDUCATION', 'MARRIAGE', 'AGE', 'BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']]

# Prevendo os valores para PAY_1
df_pay1_na['PAY_1'] = rf_imput.predict(df_pay1_na.values)

### Analisando os resultados (impacto no modelo)

In [ ]:
# Pegando os valores da variavel dependente
na_pay1 = df_clean_1['PAY_1'] == 'Not available'
defaut_na_pay = df_clean_1.loc[na_pay1,:]
defaut = defaut_na_pay['default payment next month']

x = df_clean_3[['LIMIT_BAL', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']]

y = df_clean_3['default payment next month']

# Fazendo uma nova separação dos dados sem os valore
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y,random_state=1, test_size=0.2)

# Fazer uma nova separação de dados com os valores previstos
x_fill_pay1_treino,x_fill_pay1_teste,y_fill_pay1_treino, y_fill_pay1_teste = train_test_split(df_pay1_na,defaut, test_size=0.2, random_state=1)

# Combinando os dados 
x_treino_completo = np.concatenate((x_treino, x_fill_pay1_treino), axis=0)
y_treino_completo = np.concatenate((y_treino, y_fill_pay1_treino), axis=0)
x_teste_completo = np.concatenate((x_teste, x_fill_pay1_teste), axis=0)
y_teste_completo = np.concatenate((y_teste, y_fill_pay1_teste), axis=0)

In [ ]:
# Fazendo a analise de resultado 
#tem que ter resultado proximo ao que foi feito anteriormente, quando foi encontrado os melhores parametros da Arvore de decisão
inputation_compare_cv = cross_validate(rf,x_treino_completo, y_treino_completo, scoring='roc_auc', cv =4 , n_jobs=1,verbose=1, return_train_score=True, 
                                       return_estimator=True, error_score='raise')
np.mean(inputation_compare_cv['test_score'])

# o resultado está bem proximo ao que foi treinado anteriormente... portanto, não houve queda ao imputar os novos valores...

# Treinando o modelo final

In [184]:
# Modelo com os melhores parametros encontrado 
rf = RandomForestClassifier(max_depth=12, n_estimators=100)

# Treinando com todos as observações do dataset
rf.fit(x_treino_completo,y_treino_completo)

# Metricas
y_teste_pred_prob = rf.predict_proba(x_teste_completo)
roc_auc_score(y_teste_completo, y_teste_pred_prob[:,1])

# Modelo sem over e under fit. Resultado como esperado. Modelo robusto e pronto para o uso.

0.7634087763033293